## Depedencies

In [1]:
from get_processed_data import *
import os
import requests
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

2025-03-01 02:20:06.319313: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 02:20:06.324226: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 02:20:06.339771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-01 02:20:06.371411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-01 02:20:06.371475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 02:20:06.391942: I tensorflow/core/platform/cpu_feature_guard.cc:

## Get Data

In [2]:
# Obtener los datos planteados para el ejercicio. Si se desea el detalle de las
# funciones, ver el archivo "get_processed_data"

# Nota: El codigo se ejecuto para guardar el archivo en una ruta definida, no obstante,
# podra ejecutarlo para validar la funcionalidad

df = get_data()
num_f = numerical_feature_selection(df,quantitative_variables,target_value)
cat_f = categorical_feature_selection(df,categorical_variables,target_value)
df_processed = pd.concat([num_f , cat_f],axis=1)
df_processed.columns = df_processed.columns.str.lower()
df_processed.columns = df_processed.columns.str.lower().str.strip().str.replace(" ", "_")
df_processed.to_csv("./data/processed_data/df_processed.csv",index=False)

Dataframe was loaded
---------------------------
Selected Numerical Values
                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology   False
4      Vertical_Distance_To_Hydrology   False
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am   False
7                      Hillshade_Noon   False
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True


/app/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


---------------------------
Selected Categorical Values
                      Columns  Retain
0       Wilderness_Area_Cache    True
1   Wilderness_Area_Commanche   False
2       Wilderness_Area_Neota   False
3       Wilderness_Area_Rawah   False
4             Soil_Type_C2702   False
5             Soil_Type_C2703    True
6             Soil_Type_C2704    True
7             Soil_Type_C2705    True
8             Soil_Type_C2706   False
9             Soil_Type_C2717    True
10            Soil_Type_C3501   False
11            Soil_Type_C3502   False
12            Soil_Type_C4201   False
13            Soil_Type_C4703    True
14            Soil_Type_C4704   False
15            Soil_Type_C4744   False
16            Soil_Type_C4758   False
17            Soil_Type_C5101   False
18            Soil_Type_C5151   False
19            Soil_Type_C6101   False
20            Soil_Type_C6102   False
21            Soil_Type_C6731   False
22            Soil_Type_C7101   False
23            Soil_Type_C7102   

OSError: Cannot save file into a non-existent directory: 'data/processed_data'

## Pipeline

### Parameters

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'
# directory of the raw data files
_data_root = './data/processed_data'
# path to the raw training data
_data_filepath = os.path.join(_data_root, 'processed_data.csv')

## Download data
os.makedirs(_data_root, exist_ok=True)

context = InteractiveContext(pipeline_root=pipeline_root)

### Generate Examples

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=data_root)

In [ ]:
# Execute the component
context.run(example_gen)

### Statistics

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

In [3]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

NameError: name 'context' is not defined

# Metadata

## Automatic learning Metadata

In [ ]:
# Path to metadata storage
metadata_connection_config = context.metadata_connection_config

# MetadataStore creation
store = metadata_store.MetadataStore(metadata_connection_config)

#showing all artifact types
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)

In [ ]:
# Retreiving 'Schema' artifacts
schema_artifacts = store.get_artifacts_by_type('Schema')

# Showing details for Schema artifacts
for artifact in schema_artifacts:
    print(f"Artifact ID: {artifact.id}")
    print(f"URI: {artifact.uri}")

## Retreiving main artifacts

In [ ]:
# retreiving main artifacts for 'Examples'
Examples = get_artifacts_details(store, 'Examples')
Examples

In [ ]:
# retreiving main artifacts for 'Schema'
schema = get_artifacts_details(store, 'Schema')
schema